In [178]:
import pandas as pd
import numpy as np

download_path = '~/Downloads/OHASDataset.csv'

def load_csv(path):
    return pd.read_csv(path)

data = load_csv(download_path)


In [179]:
data.describe()

,Weight,Height,Age,BMI_Level
count,2129.000000,2129.000000,2129.000000,2129.000000
mean,263.462189,172.744481,25.689056,30.451574
std,366.432493,11.167576,6.185196,6.155878
min,42.000000,140.000000,14.000000,15.960000
25%,92.000000,162.000000,21.000000,25.935000
50%,145.000000,172.000000,25.000000,30.210000
75%,290.000000,181.000000,29.000000,34.320000
max,3363.000000,213.000000,57.000000,53.130000


In [180]:
data.dtypes

Disease         object
Disease_CUI     object
Symptoms        object
Symptom_CUI     object
Weight           int64
Height           int64
Intensity       object
Severity        object
Age              int64
Gender          object
BMI_Level      float64
Region          object
Season          object
dtype: object

In [181]:
#I don't see Weight or Height being useful because BMI is simply a combonation of those factors
#so I am going to simply remove them
data.drop(['Height'], axis=1)
data.drop(['Weight'], axis=1)

,Disease,Disease_CUI,Symptoms,Symptom_CUI,Height,Intensity,Severity,Age,Gender,BMI_Level,Region,Season
0,influenza,C0162565,uncoordi162tion,C0039239,180,high,medium,24,female,27.900,southwest,Summer
1,influenza,C0162565,fever,C0000737,170,low,medium,23,male,33.770,southeast,Summer
2,influenza,C0162565,pleuritic pain,C0235704,162,low,low,24,male,33.000,southeast,Summer
3,influenza,C0162565,snuffle,C0030554,162,high,medium,34,male,22.705,northwest,Summer
4,influenza,C0162565,throat sore,C0030552,185,low,high,21,male,28.880,northwest,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...
2124,migraine disorders,C0020532,dizziness,C0005959,162,high,high,20,male,38.170,northwest,Summer
2125,migraine disorders,C0020532,numbness,C0031256,173,low,low,20,female,36.860,northeast,Summer
2126,migraine disorders,C0020532,162usea,C0014591,168,high,high,24,female,32.395,southwest,Winter
2127,migraine disorders,C0020532,fever,C0025323,168,medium,medium,17,male,42.750,southeast,Winter


In [182]:
#Delete uncesessary/redundant rows
data.drop(['Disease_CUI'], axis=1, inplace=True)
data.drop(['Symptom_CUI'], axis=1, inplace=True)
data.drop(['Region'], axis=1, inplace=True)
#majority of cases both severity and intensity hold the same value, so may aswell delete one of them...
data.drop(['Severity'], axis=1, inplace=True)

In [183]:
#drop unfilled rows
#data = data.replace('', np.nan)
#data = data.dropna()
#data.drop(index=225)

In [184]:
#We need to convert all of the text attributes to xcode
from sklearn.preprocessing import OneHotEncoder

def convert_to_onehot(x):
    y = x.reshape(-1,1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(y)
    return enc.transform(y).toarray()

In [240]:
#get all symptom values then convert to onehot(pog)
symp = [x for x in data["Symptoms"].values]
sym = np.array(vals)
sy = convert_to_onehot(val)
#Do the same for the rest of the text attributes
dise = [x for x in data["Disease"].values]
dis = np.array(dise)
de = convert_to_onehot(dis)
intens = [x for x in data["Intensity"].values]
inten = np.array(intens)
inte = convert_to_onehot(inten)
gend = [x for x in data["Gender"].values]
gen = np.array(gend)
ge = convert_to_onehot(gen)
#other row values
age = data["Age"].values
BMI = data["BMI_Level"].values
height = data["Height"].values
#Labels
disease = data["Disease"].values
dis = []
for i in range(0,2000):
    if i == 225: 
        pass
    else:     
        dis.append(disease[i])
diseases = np.array(dis)

In [242]:
#Create a list with all attributes
nLabel = np.array([])

for i in range(0, 2000):
    if i == 225:
        pass
    else:
        notLabel = np.concatenate([ge[1],sy[1],inte[1],age[1].ravel(),BMI[1].ravel(),height[1].ravel()])
        
    np.append(nLabel, notLabel) 

print(nLabel)

[]


In [239]:
#Split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nLabel, diseases, test_size=0.33, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [826000, 2129]

In [211]:
#First model testing
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=2, random_state=0)

regr.fit(X_train.reshape(-1,1),  y_train.reshape(-1,1))

ValueError: setting an array element with a sequence.